In [30]:
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.signal import welch
import pandas as pd
import seaborn
import numpy as np
from matplotlib.colors import LogNorm
import math 
#<......necesario
Samples = 201

In [31]:
filename = "/home/natalia/Escritorio/ciencia-de-datos/tp-2/P01.mat"

In [32]:
p01 = sio.loadmat(filename)

In [33]:
matrix_p01 = p01['data']

print matrix_p01.shape

#epoch x electrodos x muestras

cant_epochs = matrix_p01.shape[0]
cant_electrodos = matrix_p01.shape[1]
cant_muestras = matrix_p01.shape[2] 
#SAMPLES

(894, 256, 201)


In [48]:
#electrodo 8
electrodo_8_matrix = matrix_p01[:, 8, :]
#electrodo 44
electrodo_44_matrix = matrix_p01[:,44, :]
electrodo_80_matrix = matrix_p01[:, 80, :]
electrodo_131_matrix = matrix_p01[:, 131, :]
electrodo_185_matrix = matrix_p01[:, 185, :]
#print electrodo_8_matrix.shape
#print electrodo_44_matrix.shape

In [35]:
#Tamaño de un bin = (max(x)-min(x)) * (2*(x.75-x.25)^-1) * n^{1/3}

#x.75 y x.25 son los cuartiles de la mustra, es decir los valores que acumulan .25 y .75 de probabilidad
#suponindo que tomamos a todos los elementos de la matriz con igual probabilidad 1/(#elementos)
#los cuartiles surgen de ordenar los elementos 
#y tomar el que deje 1/4 de probabilidad(#elementos) a la derecha = x25
#y el que deje 1/4 de probabilidad(#elementos) a la izquierda = x75

In [43]:
def calculate_bin_max_min(matrix_sujeto, max_val, min_val,t_bin):
    x25_parcial = np.zeros(cant_electrodos)
    x75_parcial = np.zeros(cant_electrodos)
    max_parcial = np.zeros(cant_electrodos)
    min_parcial = np.zeros(cant_electrodos)
    
    #fila y columna de la matriz: electrodo_actual<<
    tam_fila = cant_epochs
    tam_columna = cant_muestras

    n_val = tam_fila*tam_columna

    f_x75 = int((n_val*0.75) % tam_fila)
    c_x75 = int((n_val*0.75) / tam_fila)
    f_x25 = int((n_val*0.25) % tam_fila)
    c_x25 = int((n_val*0.25) / tam_fila)

    for i in range(0,cant_electrodos):
        electrodo_actual = matrix_sujeto[:, i, :]

        max_parcial[i] = electrodo_actual.max()
        min_parcial[i] = electrodo_actual.min()
        
        electrodo_actual.sort(axis=0)

        x75_parcial[i] = electrodo_actual[f_x75][c_x75]
        x25_parcial[i] = electrodo_actual[f_x25][c_x25]
        
    x25 = np.mean(x25_parcial)
    x75 = np.mean(x75_parcial)
    max_val = np.mean(max_parcial)
    min_val = np.mean(min_parcial)
    
    t_bin = 2 * (x75-x25) * (math.pow(n_val,1/3))

    return max_val,min_val,t_bin
    #print (max_val-min_val)/t_bin

In [44]:
max_val,min_val,t_bin = calculate_bin_max_min(matrix_p01, 0, 0,0)
print max_val
print min_val
print t_bin

8.69419076427e-05
-7.94926540323e-05
1.97052410811e-05


In [38]:
#print electrodo_8_matrix_sort

In [39]:
#cantidad de bins
#print (max_8-min_8)/t_bin

In [40]:
def calculate_serie(electrodo_matrix, electrodo_serie, min_val, t_bin):
    tam_fila = electrodo_matrix.shape[0] #cant_epochs
    tam_columna = electrodo_matrix.shape[1] #cant_muestras

    vec = np.zeros(9)
    a = 0

    for i in range(0,tam_fila):
        for j in range(0,tam_columna):
            if ((electrodo_matrix[i][j] < (min_val+t_bin))):
                vec[0]=vec[0]+1
                electrodo_serie[i][j] = ord('A')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+2*t_bin)) & (electrodo_matrix[i][j] >= (min_val+t_bin))):
                vec[1]=vec[1]+1
                electrodo_serie[i][j] = ord('B')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+3*t_bin)) & (electrodo_matrix[i][j] >= (min_val+2*t_bin))):
                vec[2]=vec[2]+1
                electrodo_serie[i][j] = ord('C')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+4*t_bin)) & (electrodo_matrix[i][j] >= (min_val+3*t_bin))):
                vec[3]=vec[3]+1
                electrodo_serie[i][j] = ord('D')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+5*t_bin)) & (electrodo_matrix[i][j] >= (min_val+4*t_bin))):
                vec[4]=vec[4]+1
                electrodo_serie[i][j] = ord('E')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+6*t_bin)) & (electrodo_matrix[i][j] >= (min_val+5*t_bin))):
                vec[5]=vec[5]+1
                electrodo_serie[i][j] = ord('F')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+7*t_bin)) & (electrodo_matrix[i][j] >= (min_val+6*t_bin))):
                vec[6]=vec[6]+1
                electrodo_serie[i][j] = ord('G')
                a=a+1
            if ((electrodo_matrix[i][j] < (min_val+8*t_bin)) & (electrodo_matrix[i][j] >= (min_val+7*t_bin))):
                vec[7]=vec[7]+1
                electrodo_serie[i][j] = ord('H')
                a=a+1
            if (electrodo_matrix[i][j] >= (min_val+8*t_bin)):
                vec[8]=vec[8]+1
                electrodo_serie[i][j] = ord('I')
                a=a+1
    print vec
    print (tam_fila*tam_columna)-a

In [49]:
electrodo_8_serie = np.zeros((cant_epochs, cant_muestras))
electrodo_44_serie = np.zeros((cant_epochs, cant_muestras))
electrodo_80_serie = np.zeros((cant_epochs, cant_muestras))
electrodo_131_serie = np.zeros((cant_epochs, cant_muestras))
electrodo_185_serie = np.zeros((cant_epochs, cant_muestras))

print min_val
print t_bin

calculate_serie(electrodo_8_matrix,electrodo_8_serie, min_val, t_bin)
calculate_serie(electrodo_44_matrix,electrodo_44_serie, min_val, t_bin)
calculate_serie(electrodo_80_matrix,electrodo_80_serie, min_val, t_bin)
calculate_serie(electrodo_131_matrix,electrodo_131_serie, min_val, t_bin)
calculate_serie(electrodo_185_matrix,electrodo_185_serie, min_val, t_bin)

-7.94926540323e-05
1.97052410811e-05
[     0.    111.   2643.  84295.  90288.   2247.    110.      0.      0.]
0
[  3.10000000e+01   8.80000000e+01   1.78700000e+03   8.37260000e+04
   9.24570000e+04   1.57900000e+03   2.60000000e+01   0.00000000e+00
   0.00000000e+00]
0
[  0.00000000e+00   6.00000000e+01   1.81800000e+03   8.24890000e+04
   9.38720000e+04   1.44900000e+03   6.00000000e+00   0.00000000e+00
   0.00000000e+00]
0
[  0.00000000e+00   1.07000000e+02   2.06700000e+03   8.42770000e+04
   9.12400000e+04   1.95300000e+03   4.40000000e+01   6.00000000e+00
   0.00000000e+00]
0
[  6.00000000e+00   2.12000000e+02   2.97000000e+03   8.38400000e+04
   8.99960000e+04   2.54000000e+03   1.30000000e+02   0.00000000e+00
   0.00000000e+00]
0


In [46]:
electrodo_8_serie

array([[ 67.,  67.,  67., ...,  67.,  67.,  66.],
       [ 68.,  68.,  68., ...,  67.,  67.,  67.],
       [ 68.,  68.,  68., ...,  67.,  67.,  67.],
       ..., 
       [ 70.,  70.,  70., ...,  70.,  70.,  70.],
       [ 70.,  70.,  70., ...,  70.,  70.,  71.],
       [ 70.,  70.,  70., ...,  71.,  71.,  71.]])